# Google collab

In [4]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

# Генерация датасета

In [11]:
!mkdir -p dataset
!unzip /content/drive/MyDrive/dataset.zip -d dataset

�������� ��� 䠩� -p 㦥 �������.
�訡�� �� �६� ��ࠡ�⪨: -p.
�������� ��� 䠩� dataset 㦥 �������.
�訡�� �� �६� ��ࠡ�⪨: dataset.
"unzip" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.


In [2]:
import glob
import os
import random
from tqdm import tqdm
import shutil
import cv2
from ultralytics import YOLOv10

In [2]:
path_to_dataset = 'dataset'

In [3]:
#!rm -rf train/ val/
for path in ['train', 'val']:
    os.makedirs(os.path.join('images', path), exist_ok=True)
    os.makedirs(os.path.join('labels', path), exist_ok=True)

In [3]:
val_prob = 0.1
for jpg in tqdm(glob.glob(os.path.join(path_to_dataset, '*.jpg'))):
    folder = 'train' if random.random() > val_prob else 'val'
    shutil.copyfile(jpg, os.path.join('images', folder, os.path.basename(jpg)))
for jpg in tqdm(glob.glob(os.path.join(path_to_dataset, '*.txt'))):
    folder = 'train' if random.random() > val_prob else 'val'
    shutil.copyfile(jpg, os.path.join('labels', folder, os.path.basename(jpg)))

100%|██████████| 1163/1163 [00:01<00:00, 1040.23it/s]


In [3]:
!cat ./svarka.yaml

"cat" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.


# Обучение

In [6]:
!pip install ultralytics


In [1]:
from ultralytics import YOLOv10

model = YOLOv10('./yolov10x.pt')

c:\Users\witcheredd\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
results = model.train(data='svarka.yaml', epochs=20, imgsz=640, batch=1)

New https://pypi.org/project/ultralytics/8.2.32 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.34 🚀 Python-3.10.0 torch-2.3.1+cu118 CUDA:0 (NVIDIA GeForce GTX 1050 Ti, 4096MiB)
engine\trainer: task=detect, mode=train, model=./yolov10x.pt, data=svarka.yaml, epochs=20, time=None, patience=100, batch=1, imgsz=640, save=True, save_period=-1, val_period=1, cache=False, device=None, workers=8, project=None, name=train12, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, 

train: Scanning C:\Users\witcheredd\Downloads\labels\train.cache... 1161 images, 140 backgrounds, 0 corrupt: 100%|██████████| 1161/1161 [00:00<?, ?it/s]
val: Scanning C:\Users\witcheredd\Downloads\labels\val.cache... 136 images, 249 backgrounds, 0 corrupt: 100%|██████████| 373/373 [00:00<?, ?it/s]


Plotting labels to runs\detect\train12\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001111, momentum=0.9) with parameter groups 185 weight(decay=0.0), 198 weight(decay=0.0005), 197 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train12
Starting training for 20 epochs...

      Epoch    GPU_mem     box_om     cls_om     dfl_om     box_oo     cls_oo     dfl_oo  Instances       Size


       1/20      1.92G      2.419      3.447      1.853      2.178      4.008      1.566          9        640: 100%|██████████| 1161/1161 [14:08<00:00,  1.37it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 187/187 [00:45<00:00,  4.08it/s]


                   all        373        426     0.0933      0.126     0.0449     0.0129

      Epoch    GPU_mem     box_om     cls_om     dfl_om     box_oo     cls_oo     dfl_oo  Instances       Size


...

      20/20      1.91G      1.403     0.9459      1.236      1.386     0.8673      1.141          1        640: 100%|██████████| 1161/1161 [14:37<00:00,  1.32it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 187/187 [00:46<00:00,  4.05it/s]

                   all        373        426      0.307       0.56      0.286      0.136



20 epochs completed in 5.083 hours.
Optimizer stripped from runs\detect\train12\weights\last.pt, 64.1MB
Optimizer stripped from runs\detect\train12\weights\best.pt, 64.1MB

Validating runs\detect\train12\weights\best.pt...
Ultralytics YOLOv8.1.34 🚀 Python-3.10.0 torch-2.3.1+cu118 CUDA:0 (NVIDIA GeForce GTX 1050 Ti, 4096MiB)
YOLOv10x summary (fused): 503 layers, 31593710 parameters, 0 gradients, 169.8 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 187/187 [00:42<00:00,  4.36it/s]


                   all        373        426      0.308      0.561      0.286      0.136
                   adj        373        183      0.298       0.36      0.223     0.0858
                   int        373         42       0.29      0.452       0.22     0.0939
                   geo        373        135      0.341      0.696      0.345      0.192
                   pro        373         41      0.396      0.537      0.337      0.157
                   non        373         25      0.214       0.76      0.307      0.151
Speed: 0.4ms preprocess, 111.6ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to runs\detect\train12


In [3]:
results

NameError: name 'results' is not defined

In [23]:
import matplotlib.pyplot as plt
from matplotlib import patches

image = cv2.imread('images/train/1 (1).jpg')
results = model.predict(source='images/train/1 (1).jpg', conf=0.02)
fig, ax = plt.subplots(figsize=(15,10))

ax.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

for box in results[0].boxes:
    x, y, w, h = box.xywh[0]
    class_name = box.id

    rect = patches.Rectangle((x, y), w, h, linewidth=1,
    edgecolor='r', facecolor='none')

    ax.add_patch(rect)

    ax.text(x, y, class_name, fontsize=12, color='r')

plt.show()


image 1/1 c:\Users\witcheredd\Downloads\images\train\1 (1).jpg: 384x640 3 adjs, 3 geos, 88.1ms
Speed: 5.0ms preprocess, 88.1ms inference, 18.0ms postprocess per image at shape (1, 3, 384, 640)


TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.